# WhoWasWhen generator
This script is to create the database for the WhoWasWhen workflow, starting from a google spreadsheet
it can be (it used to be actually) simplified to use a tab-delimited local file, but since my main file is a google sheet I didn't want to export and save all the time
the script creates a JSON object with keys = year and values a nested dict with all the rulers
Sunny ☀️   🌡️+76°F (feels +76°F, 32%) 🌬️↘6mph 🌘&m Sat Jun  1 11:17:32 2024
W22Q2 – 153 ➡️ 212 – 21 ❇️ 343

# Setting up

In [1]:
# home: use the 3.10.9 Python environment
import pickle
import json
import sqlite3
import gspread
import gspread.exceptions
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
from time import time
from config import KEYFILE, log, GSHEET_URL, MY_PERIOD_SHEET, MY_RULERS_SHEET, MY_DB

## GetSheet
A function to retrieve tables from a google sheet and convert them into a nested dictionary
(The conversion might not be needed in the new version using sqlite)

In [2]:
def getSheet(keyfile,mysheetURL, myPeriodSheet, myColumns):
    def list_to_nested_dict(all_values, selected_columns):
        headers = all_values[0]  # The first row is the header
        indices = [headers.index(col) for col in selected_columns if col in headers]  # Indices of selected columns
        data_dict = {}
        
        for row in all_values[1:]:
            # Create a dictionary for each row with selected columns only
            row_dict = {headers[i]: row[i] for i in indices}
            # Use the first column as the key for the main dictionary
            data_dict[row[0]] = row_dict
        
        return data_dict

    scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]
    credentials = ServiceAccountCredentials.from_json_keyfile_name(keyfile, scopes) 
    file = gspread.authorize(credentials) # authenticate the JSON key with gspread
    
    # opening the file
    try:
        sheet = file.open_by_url(mysheetURL)
    
    except gspread.exceptions.NoValidUrlKeyFound as e:
        log ("URL not valid")
    except ValueError as e:
        log ("ValueError")
    except IOError as e:
    # Handle input/output errors
        log ("IOError")
    except Exception as e:
    # Catch any remaining errors
        log ("Exception")

    
    # fetching the worksheet
    try:
        worksheet = sheet.worksheet(myPeriodSheet) 
    
    except gspread.exceptions.WorksheetNotFound as e:
        log ("Sheet Not Found")


    # Get all values from the worksheet
    all_values = worksheet.get_all_values()

   
    # convertint into a dictionary with key = counter and value = all values
    data_dict = list_to_nested_dict(all_values, myColumns)
    

    return data_dict

## Populating tables

### Parse period

In [3]:
def parse_period(years):
# Function to expand the years field (by ChatGPT)
    if '-' in years:
        start_year, end_year = years.split('-')
        
        # Convert start year
        if 'BC' in start_year:
            start_year = -int(start_year.replace('BC', ''))
        else:
            start_year = int(start_year)

        # Convert end year
        if 'BC' in end_year:
            end_year = -int(end_year.replace('BC', ''))
        else:
            if 'AD' in end_year:
                end_year = int(end_year.replace('AD', ''))
            elif len(end_year) <= 2:
                # Handle short form end year like '95' in '1981-95'
                end_year = int(f"{str(start_year)[:-len(end_year)]}{end_year}")
            else:
                end_year = int(end_year)
        

        
    else:
        if 'BC' in years:
            start_year = end_year = -int(years.replace('BC', ''))
        elif 'AD' in years:
            start_year = end_year = -int(years.replace('AD', ''))
        else:
            start_year = end_year = int(years)
    return start_year, end_year

### populateTables function 

In [16]:
def populateTables(myData, db_name):

    def creatingTables():
        
        # Drop tables if they exist
        cursor.execute('DROP TABLE IF EXISTS titles;')
        cursor.execute('DROP TABLE IF EXISTS years;')
        cursor.execute('DROP TABLE IF EXISTS byPeriod;')
        cursor.execute('DROP TABLE IF EXISTS byYear;')


        # Create the tables
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS titles (
            titleID INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT UNIQUE,
            maxCount INTEGER,
            titlePlural TEXT
        );
        ''')

        cursor.execute('''
        CREATE TABLE IF NOT EXISTS years (
            yearID INTEGER PRIMARY KEY AUTOINCREMENT,
            year INTEGER UNIQUE
        );
        ''')


        cursor.execute('''
        CREATE TABLE IF NOT EXISTS byPeriod (
            periodID INTEGER PRIMARY KEY AUTOINCREMENT,
            rulerID INTEGER,
            titleID INTEGER,
            progrTitle INTEGER,
            period TEXT,
            startYear INTEGER,
            endYear INTEGER,
            notes TEXT,
                FOREIGN KEY (rulerID) REFERENCES rulers (rulerID),
                FOREIGN KEY (titleID) REFERENCES titles (titleID)
            );
        ''')


        cursor.execute('''
        CREATE TABLE IF NOT EXISTS byYear (
            yearID INTEGER,
            periodID INTEGER,
                FOREIGN KEY (yearID) REFERENCES years (yearID),
                FOREIGN KEY (periodID) REFERENCES byPeriod (periodID),
                PRIMARY KEY (yearID, periodID)
        );
        ''')
    def fetchTitleID(title):
        # Function to get the titleID for a given title
        cursor.execute('''
            SELECT titleID FROM titles WHERE title = ?
        ''', (title,))
        title_id = cursor.fetchone()
        return title_id[0] if title_id else None

    def insert_title(title):
    # Function to insert a unique title and get its titleID
        cursor.execute('''
            INSERT OR IGNORE INTO titles (title)
            VALUES (?)
            ''', (title,))
        conn.commit()
        

        # Retrieve the titleID for the given title
        title_id = fetchTitleID(title)
        
        return title_id if title_id else None

    def populate_byPeriod (rulerID, titleID, progrTitle, period, startYear, endYear,notes):

        # Insert each period in the byPeriod table and get the periodID
        cursor.execute('''
            INSERT INTO byPeriod (rulerID, titleID, progrTitle, period, startYear, endYear,notes)
            VALUES (?, ?, ?, ?, ?, ?,?)
            ''', (rulerID, titleID, progrTitle, period, startYear, endYear,notes))
        conn.commit()    
        return cursor.lastrowid

    def insert_year(year):
    # Function to insert unique year and get its yearID
        cursor.execute('''
        INSERT OR IGNORE INTO years (year)
        VALUES (?)
        ''', (year,))
        conn.commit()
        # Retrieve the yearID for the given year
        
        cursor.execute('''
            SELECT yearID FROM years WHERE year = ?
        ''', (year,))
        year_id = cursor.fetchone()
        
        # Return the yearID
        return year_id[0] if year_id else None

    def populateByYear (periodID, startYear, endYear):
        # Function to insert ruler title relationships for each year

        for year in range(startYear, endYear + 1):

            # Insert each year into the years table and get the yearID
            yearID = insert_year(year)


            # Insert periodID and yearID into the byYear table
            cursor.execute('''
            INSERT OR IGNORE INTO byYear (periodID, yearID)
            VALUES (?, ?)
            ''', (periodID, yearID))
        conn.commit()

    # Connect to SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Create tables
    creatingTables()
    
    # processing the dictionary
    titleCheck = {}
    for key,row in myData.items():
        title = row['Title']
        rulerID = row['RulerID']
        period = row['Period'].strip()  # e.g. "1509-1547"
        notes = row['Notes']
        startYear, endYear = parse_period(period)

        if title not in titleCheck:
            titleCheck[title] = 1
            titleID = insert_title(title)
            # Insert the title into the titles table and get its titleID
        else:
            titleCheck[title] += 1
            titleID = fetchTitleID(title)
            
        
        periodID = populate_byPeriod(rulerID, titleID, titleCheck[title], period, startYear, endYear, notes)        
        
        # Insert year and period relationships into the byYear table
        populateByYear (periodID, startYear, endYear)
        
    for myTitle in titleCheck:
        cursor.execute('''
            UPDATE titles
            SET maxCount = ?
            WHERE title = ?
        ''', (titleCheck[myTitle], myTitle))
        conn.commit()


    # adding plurals
    myPlurals = {'Pope': 'Popes', 
                 'English Monarch': 'English Monarchs', 
                 'US president': 'US presidents', 
                 'British Prime Minister': 'British Prime Ministers', 
                 'French Monarch': 'French Monarchs', 
                 'King of the West Franks': 'Kings of the West Franks', 
                 'King of the East Franks': 'Kings of the East Franks', 
                 'King of the Franks': 'Kings of the Franks', 
                 'King of France': 'Kings of France', 
                 'French Government': 'French Governments', 
                 'French Emperor': 'French Emperors', 
                 'French President': 'French Presidents', 
                 'Byzantine Emperor': 'Byzantine Emperors', 
                 'Emperor of China': 'Emperors of China', 
                 'Holy Roman Emperor': 'Holy Roman Emperors', 
                 'Emperor of the Carolingian Empire': 'Emperors of the Carolingian Empire', 
                 'Roman Emperor': 'Roman Emperors', 
                 'Roman Emperor (East)': 'Roman Emperors (East)', 
                 'Roman Emperor (West)': 'Roman Emperors (West)', 
                 'Russian Emperor': 'Russian Emperors', 
                 'Prince of Moscow': 'Princes of Moscow', 
                 'Tsar of Russia': 'Tsars of Russia', 
                 'Chairman of the Communist Party of the Soviet Union': 'Chairmen of the Communist Party of the Soviet Union', 
                 'Russian President': 'Russian Presidents',
                 'Antipope': 'Antipopes',
                 'Scottish Monarch': 'Scottish Monarchs',
                 'Neapolitan ruler': 'Neapolitan rulers',
                 'Spanish Monarch': 'Spanish Monarchs'}
    
    for myTitle in myPlurals:
        cursor.execute('''
            UPDATE titles
            SET titlePlural = ?
            WHERE title = ?
        ''', (myPlurals[myTitle], myTitle))
    conn.commit()

    log("Titles and junction table successfully exported to SQLite database")
    
    # Close the connection
    conn.close()

### Populate Rulers
Creating the `rulers` table

In [5]:
def populateRulers(myData, db_name):
    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    cursor.execute('DROP TABLE IF EXISTS rulers;')
    # Create the rulers table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS rulers (
            rulerID INTEGER PRIMARY KEY,
            name TEXT,
            personal_name TEXT,
            epithet TEXT,
            wikipedia TEXT,
            notes TEXT
        )
    """)

    # Insert data into the rulers table
    for key, row in myData.items():
        cursor.execute("""
            INSERT OR IGNORE INTO rulers (
                rulerID, name, personal_name, epithet, wikipedia, notes
            ) VALUES (?, ?, ?, ?, ?, ?)
        """, (
            int(row['RulerID']),
            row['Name'],
            row['Personal Name'],
            row['Epithet'],
            row['Wikipedia'],
            row['Notes']
        ))

    # Commit changes and close the connection
    conn.commit()
    conn.close()

    log(f"Rulers table successfully exported to SQLite database")

# 📍 Main function 

In [18]:
main_start_time = time()
# first, I need to get the 2 spreadheets from the google sheet: 1) rulers and 2) periods

# get rulers
allRulers = getSheet(KEYFILE, GSHEET_URL, MY_RULERS_SHEET, ["RulerID","Name","Personal Name","Wikipedia", "Epithet","Personal Name", "Notes"])

# get periods
selected_columns = ["Title", "RulerID", "Period", "Notes"] #columns to be fetched from the gsheet
allValues = getSheet(KEYFILE, GSHEET_URL, MY_PERIOD_SHEET, selected_columns)

main_timeElapsed = time() - main_start_time
log(f"\nscript duration: {round (main_timeElapsed,3)} seconds")
    

# exportRulersYears(allValues,allRulers)
# createIconDict()

# second, I create the sqlite tables
populateRulers(allRulers, MY_DB)

populateTables(allValues, MY_DB)




# result= {"items": [{
#     "title": "Done!" ,
#     "subtitle": "ready to use WhoWhasWhen now 👍️",
#     "arg": "",
#     "icon": {

#             "path": "icons/done.png"
#         }
#     }]}
# print (json.dumps(result))
log("Done 👍️")



script duration: 1.158 seconds
Rulers table successfully exported to SQLite database
Titles and junction table successfully exported to SQLite database
Done 👍️


# Other functions

## Extracting tables from a wikipedia page  

In [27]:
"""
This script from chatGPT will extract and parse multiple tables from a single Wikipedia page and export them into a csv file and dictionary. Used to output the roman consuls from wikipedia
"""
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Roman_consuls"

# Send a GET request to the Wikipedia page
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Parse the page content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table', {'class': 'wikitable'})

# Initialize a list to store all tables
all_tables = []

# Function to ensure unique column names
def make_unique_columns(columns):
    seen = {}
    unique_columns = []
    for col in columns:
        if col in seen:
            seen[col] += 1
            unique_columns.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            unique_columns.append(col)
    return unique_columns

# Loop through all tables and process them
for i, table in enumerate(tables):
    # Extract table headers
    headers = [header.text.strip() for header in table.find_all('th')]
    headers = make_unique_columns(headers)  # Ensure unique headers
    
    # Prepare data for rows
    rows = []
    for row in table.find_all('tr'):
        row_data = []
        row_links = []
        for cell in row.find_all(['td', 'th']):
            # Get text in the cell
            cell_text = cell.text.strip()
            # Get hyperlink if present
            link = cell.find('a')
            cell_link = f"https://en.wikipedia.org{link['href']}" if link and link.has_attr('href') else None
            # Append text and link
            row_data.append(cell_text)
            row_links.append(cell_link)
        # Append row only if there's data
        if row_data:
            rows.append((row_data, row_links))
    
    # Prepare columns for text and links
    text_data = [row[0] for row in rows]
    link_data = [row[1] for row in rows]
    
    # Create DataFrames
    df = pd.DataFrame(text_data, columns=headers if headers else [f"Column_{j+1}" for j in range(len(text_data[0]))])
    link_headers = [f"{col}_link" for col in df.columns]
    link_df = pd.DataFrame(link_data, columns=link_headers)
    
    # Combine text and link DataFrames
    combined_df = pd.concat([df, link_df], axis=1)
    combined_df['Source_Table'] = f"Table_{i+1}"  # Add a column to identify the source table
    
    # Add the combined DataFrame to the list
    all_tables.append(combined_df)

# Concatenate all tables into one DataFrame
final_df = pd.concat(all_tables, ignore_index=True)  # Ensure unique indexing

# Save the concatenated DataFrame to a single CSV file
final_df.to_csv("roman_consuls_combined.csv", index=False)

print("All tables have been saved into a single CSV file: 'roman_consuls_combined.csv'")


All tables have been saved into a single CSV file: 'roman_consuls_combined.csv'


# Older code

In [ ]:
'''
OLDER CODE TO DELETE
def exportRulers (myData):
    # Initialize the dictionary
    rulers = {}
    for key,row in myData.items():
        
        rulers [row['RulerID']] = {
            "name": row['Name'],
            "personal name": row['Personal Name'],
            "epithet": row['Epithet'],
            "wikipedia": row['Wikipedia'],
            "title": [],
            "notes": row['Notes']
        }
    
    # # Export the resulting dictionary to a JSON file
    return rulers
    

'''

In [ ]:
def exportRulersYears (myData,myRulers):
    # Initialize the dictionary
    rulerYears_dict = {}
    rulers_dict = {}
    myCounter = 0
    for key,row in myData.items():
        
        ruler_type = row['Ruler']
        myCounter += 1
       
        name = row['Name']
        years_field = row['Year'].strip()
        personalName = row['Personal Name']
        years = expand_years(years_field)
        year_max = max(years)
        year_min = min(years)

        if ruler_type in rulers_dict:
            rulers_dict[ruler_type].append({'progr': myCounter,
                                            'name': row['Name'],
                                            'personal name': row['Personal Name'],
                                            'period': row['Year'],
                                            'startYear': year_min,
                                            'endYear': year_max,
                                            'rulerID': row['RulerID']
                                            })
        else:
            # initialize ruler type
            myCounter = 1
            rulers_dict[ruler_type] = [{'progr': myCounter,
                                        'name': row['Name'],
                                        'personal name': row['Personal Name'],
                                        'period': row['Year'],
                                        'startYear': year_min,
                                        'endYear': year_max,
                                        'rulerID': row['RulerID']

                                        }]
        if row['RulerID'] in myRulers.keys():
            if row['Ruler'] not in myRulers[row['RulerID']]['title']:
                myRulers[row['RulerID']]['title'].append ((row['Ruler'],row['Year']))
            
        
        for year in years:
            year_str = str(year)
            if year_str not in rulerYears_dict:
                rulerYears_dict[year_str] = {
                    ruler_type: [{
                        'rulerID': row['RulerID'],
                        'name': name, 
                        'period': years_field, 
                        'startYear': year_min,
                        'endYear': year_max,
                        'searchString': f"{year_str} {ruler_type} {name} {personalName}".lower().strip(),
                        'personal name': personalName}]}  # Save the ruler as a list
            else:
                if ruler_type in rulerYears_dict[year_str]:
                    rulerYears_dict[year_str][ruler_type].append({
                        'rulerID': row['RulerID'],
                        'name': name, 
                        'period': years_field, 
                        'startYear': year_min,
                        'endYear': year_max,
                        'searchString': f"{year_str} {ruler_type} {name} {personalName}".lower().strip(),
                        'personal name': personalName})  # Append to the existing list
                else:
                    rulerYears_dict[year_str][ruler_type] = [{
                        'rulerID': row['RulerID'],
                        'name': name, 
                        'period': years_field,
                        'startYear': year_min,
                        'endYear': year_max,
                        'searchString': f"{year_str} {ruler_type} {name} {personalName}".lower().strip(),
                        'personal name': personalName}]  # Create a new list

    # Export the resulting dictionary to a JSON file
    with open('rulersYears.pkl', 'wb') as json_file:
        pickle.dump(rulerYears_dict, json_file)
    
    # Export the resulting dictionary to a JSON file
    with open('rulersLists.pkl', 'wb') as json_file:
        pickle.dump(rulers_dict, json_file)
    
    with open('rulersInfo.pkl', 'wb') as json_file:
        pickle.dump(myRulers, json_file)
# Export the resulting dictionary to a JSON file
    with open('rulersYears.json', 'w') as json_file:
        json.dump(rulerYears_dict, json_file, indent=4)
    
    # Export the resulting dictionary to a JSON file
    with open('rulersLists.json', 'w') as json_file:
        json.dump(rulers_dict, json_file, indent=4)
    
    with open('rulersInfo.json', 'w') as json_file:
        json.dump(myRulers, json_file, indent=4)

In [ ]:
# Function to expand the years field (by ChatGPT)
def expand_years(years):
    if '-' in years:
        start_year, end_year = years.split('-')
        
        # Convert start year
        if 'BC' in start_year:
            start_year = -int(start_year.replace('BC', ''))
        else:
            start_year = int(start_year)

        # Convert end year
        if 'BC' in end_year:
            end_year = -int(end_year.replace('BC', ''))
        else:
            if 'AD' in end_year:
                end_year = int(end_year.replace('AD', ''))
            elif len(end_year) <= 2:
                # Handle short form end year like '95' in '1981-95'
                end_year = int(f"{str(start_year)[:-len(end_year)]}{end_year}")
            else:
                end_year = int(end_year)

        # Handle wraparounds around centuries
        if start_year > end_year:
            if start_year > 0 and end_year < 0:
                # BC to AD wraparound
                return list(range(start_year, 0)) + list(range(1, end_year + 1))
            elif start_year < 0 and end_year > 0:
                # AD to BC wraparound
                return list(range(start_year, 1)) + list(range(-1, end_year - 1, -1))
            else:
                return list(range(start_year, end_year - 1, -1))
        else:
            return list(range(start_year, end_year + 1))
    else:
        if 'BC' in years:
            return [-int(years.replace('BC', ''))]
        elif 'AD' in years:
            return [int(years.replace('AD', ''))]
        else:
            return [int(years)]

In [ ]:
def createRulersSearch (data,searchFields=['personal name','name']):
    rulers_search = {}

    for key,row in data.items(): #year level
        rulers_search[key] = {}
        for title, value in row.items(): #ruler level
            for mytitle in value: #list of rulers
                titleStringList = []
                titleStringList.append (key.casefold())
                titleStringList.append (title.casefold())
                for key2, value2, in mytitle.items(): #multiple ites per ruler
                    if key2 in searchFields and value2:
                        titleStringList.append (value2.casefold())
                
                rulers_search[key][title] = (" ".join(titleStringList))
                    

    # Export the resulting dictionary to a JSON file
    with open('rulersSearch.json', 'w') as json_file:
        json.dump(rulers_search, json_file, indent=4)

In [29]:
import re

def is_number_like(term):
    """
    Check if the term is a number, BC year, or a range.
    Valid formats:
    - Single year (e.g., 20 or -20)
    - Year range (e.g., -20--10, -20-10, 20-40)
    """
    # Regex for single year or valid range
    pattern = r"^-?\d+$|^-?\d+-\-?\d+$"
    return bool(re.match(pattern, term))

# Sample search terms
terms = ["20", "-20", "-20-10", "-20--10", "20-40", "random", "abc-def"]

# Check if each term is a number or range
results = {term: is_number_like(term) for term in terms}

# Print results
print(results)


{'20': True, '-20': True, '-20-10': True, '-20--10': True, '20-40': True, 'random': False, 'abc-def': False}


In [31]:
"-20".isdigit()

False